%%

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

%%

In [ ]:
train_path = "data/KDDTrain+.csv"
test_path  = "data/KDDTest+.csv"

In [ ]:
col_names = [
    'duration','protocol_type','service','flag','src_bytes','dst_bytes','land',
    'wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised',
    'root_shell','su_attempted','num_root','num_file_creations','num_shells',
    'num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count',
    'srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate',
    'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count',
    'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate',
    'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','label','difficulty'
]

%%

In [ ]:
train_df = pd.read_csv(train_path, names=col_names)
test_df  = pd.read_csv(test_path, names=col_names)
train_df.drop('difficulty', axis=1, inplace=True)
test_df.drop('difficulty', axis=1, inplace=True)

Simplify labels

In [ ]:
train_df['label'] = train_df['label'].apply(lambda x: 'normal' if x == 'normal' else 'attack')
test_df['label']  = test_df['label'].apply(lambda x: 'normal' if x == 'normal' else 'attack')

Encode label

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])
y_test  = label_encoder.transform(test_df['label'])

In [ ]:
X_train = train_df.drop('label', axis=1)
X_test  = test_df.drop('label', axis=1)

%%<br>
Define categorical and numerical columns

In [ ]:
cat_cols = ['protocol_type', 'service', 'flag']
num_cols = [c for c in X_train.columns if c not in cat_cols]

Preprocessing: OneHotEncode categorical + StandardScale numeric

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

Build the pipeline

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier(n_jobs=-1))
])

%%<br>
Define parameter grid for GridSearch

In [ ]:
params = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

Grid Search with Pipeline

In [ ]:
grid = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='f1_macro', n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

%%<br>
Evaluate best model

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nBest Parameters:", grid.best_params_)
print("Best CV Score (F1-macro):", grid.best_score_)

%%<br>
Save everything

In [ ]:
os.makedirs("models", exist_ok=True)
joblib.dump(grid.best_estimator_, "models/knn_pipeline.pkl")
joblib.dump(label_encoder, "models/label_encoder.pkl")

In [ ]:
print("\nFull pipeline and label encoder saved successfully to ./models/")